# Specialist Search Agent (SSA) – RAG Pipeline (V2)
This notebook implements an enhanced Referral Matching Agent using a hybrid approach:
- FAISS-based semantic retrieval
- Rule-based specialty detection
- Composite scoring for ranking candidates
- Reflection loop for robustness in low-confidence scenarios


# Import Libraries

In [1]:
!pip install langchain faiss-cpu sentence-transformers -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [1]:
import pandas as pd
import json
import random  # Used to simulate real-time availability changes

# Import LangChain components for FAISS vector store and embeddings.
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Data Loading

In [2]:
# Load the CSV file containing provider metadata.
df = pd.read_csv("Mock_Specialist_Dataset.csv")
print("Provider Metadata Sample:")
print(df.head())

# Print column names for debugging.
print("CSV Columns:")
print(df.columns)

Provider Metadata Sample:
   specialist_id                 name          specialty        location  \
0              1      Dr. Aarti Mehta       Cardiologist   New Haven, CT   
1              2     Dr. James Wright      Pulmonologist    Stamford, CT   
2              3     Dr. Leena Kapoor        Neurologist    Hartford, CT   
3              4       Dr. Brian Choi  Internal Medicine  Bridgeport, CT   
4              5  Dr. Sophia Martinez       Cardiologist     Norwalk, CT   

  insurance_accepted  availability_next_3_days  \
0   BlueCross, Aetna                         3   
1       Aetna, Cigna                         0   
2  United, BlueCross                         2   
3    Medicare, Aetna                         5   
4  BlueCross, United                         1   

                                     profile_summary  
0  Board-certified cardiologist with 10 years of ...  
1  Specialist in respiratory and lung disorders i...  
2  Neurodiagnostics expert focusing on dizziness,..

# Data preparation

In [3]:
# Prepare texts and metadata for each provider.
texts = []
metadatas = []

for idx, row in df.iterrows():
    # Build a text description using the correct (lowercase) CSV column names.
    doc_text = (
        f"Specialist profile for {row['name']}. "
        f"Specialty: {row['specialty']}. "
        f"Location: {row['location']}. "
        f"Insurance accepted: {row['insurance_accepted']}. "
        f"Availability next 3 days: {row['availability_next_3_days']}. "
        f"Profile summary: {row['profile_summary']}"
    )
    texts.append(doc_text)
    metadatas.append(row.to_dict())


In [4]:
texts

['Specialist profile for Dr. Aarti Mehta. Specialty: Cardiologist. Location: New Haven, CT. Insurance accepted: BlueCross, Aetna. Availability next 3 days: 3. Profile summary: Board-certified cardiologist with 10 years of experience in treating arrhythmia and heart failure.',
 'Specialist profile for Dr. James Wright. Specialty: Pulmonologist. Location: Stamford, CT. Insurance accepted: Aetna, Cigna. Availability next 3 days: 0. Profile summary: Specialist in respiratory and lung disorders including COPD and asthma. 15+ years in pulmonary care.',
 'Specialist profile for Dr. Leena Kapoor. Specialty: Neurologist. Location: Hartford, CT. Insurance accepted: United, BlueCross. Availability next 3 days: 2. Profile summary: Neurodiagnostics expert focusing on dizziness, migraines, and stroke recovery.',
 'Specialist profile for Dr. Brian Choi. Specialty: Internal Medicine. Location: Bridgeport, CT. Insurance accepted: Medicare, Aetna. Availability next 3 days: 5. Profile summary: General pr

In [5]:
metadatas

[{'specialist_id': 1,
  'name': 'Dr. Aarti Mehta',
  'specialty': 'Cardiologist',
  'location': 'New Haven, CT',
  'insurance_accepted': 'BlueCross, Aetna',
  'availability_next_3_days': 3,
  'profile_summary': 'Board-certified cardiologist with 10 years of experience in treating arrhythmia and heart failure.'},
 {'specialist_id': 2,
  'name': 'Dr. James Wright',
  'specialty': 'Pulmonologist',
  'location': 'Stamford, CT',
  'insurance_accepted': 'Aetna, Cigna',
  'availability_next_3_days': 0,
  'profile_summary': 'Specialist in respiratory and lung disorders including COPD and asthma. 15+ years in pulmonary care.'},
 {'specialist_id': 3,
  'name': 'Dr. Leena Kapoor',
  'specialty': 'Neurologist',
  'location': 'Hartford, CT',
  'insurance_accepted': 'United, BlueCross',
  'availability_next_3_days': 2,
  'profile_summary': 'Neurodiagnostics expert focusing on dizziness, migraines, and stroke recovery.'},
 {'specialist_id': 4,
  'name': 'Dr. Brian Choi',
  'specialty': 'Internal Medi

# Vector embedding

In [6]:
# Initialize the embedding model.
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build the FAISS vector store.
vector_store = FAISS.from_texts(texts, embeddings, metadatas=metadatas)
print("Vector store created successfully.")

<ipython-input-6-2269f5955cb1>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store created successfully.


# Helper Functions

In [7]:
def extract_specialty(patient_query: str) -> str:
    """
    Rule-based extraction of the required specialty from patient symptoms.
    Returns 'Cardiologist' if key symptoms are present; otherwise, defaults to 'General Practitioner'.
    """
    query_lower = patient_query.lower()
    if "chest pain" in query_lower or "exertion" in query_lower or "dizzy" in query_lower:
        return "Cardiologist"
    return "General Practitioner"

def advanced_extract_specialty(patient_query: str) -> str:
    """
    Placeholder for a more advanced, LLM-based extraction method.
    In production, this function could use a fine-tuned model or GPT-4 to determine the specialty.
    Currently, it just calls the rule-based extractor.
    """
    # TODO: Integrate an LLM-based extraction method here.
    return extract_specialty(patient_query)

def simulate_real_time_availability(metadata: dict) -> int:
    """
    Simulates dynamic availability by modifying the 'availability_next_3_days' value.
    In a production system, this would retrieve up-to-date scheduling data from an API.
    Here, we randomly adjust the availability (ensuring it stays within [0,10]).
    """
    current_availability = metadata.get('availability_next_3_days', 0)
    # Simulate a fluctuation between -1 and +1 days (clamped to at least 0)
    simulated_change = random.choice([-1, 0, 1])
    updated_availability = max(0, min(10, current_availability + simulated_change))
    return updated_availability

def calculate_composite_score(metadata: dict, patient_location: str, patient_insurance: str, recommended_specialty: str):
    """
    Calculates a composite score for a specialist profile based on:
      - Specialty match (40%)
      - Insurance compatibility (20%)
      - Availability (20%) [uses dynamic availability]
      - Proximity (20%)

    Returns a tuple of (composite_score, score_details_dict).
    """
    # Specialty Score: Exact match yields 1; otherwise, 0.
    specialty_score = 1 if metadata.get('specialty', '').strip().lower() == recommended_specialty.lower() else 0

    # Insurance Score: Check if the patient's insurance is accepted.
    insurances = [x.strip().lower() for x in metadata.get('insurance_accepted', '').split(',')]
    insurance_score = 1 if patient_insurance.lower() in insurances else 0

    # Availability Score: Use the simulated real-time availability.
    # Lower wait time is better (score = (10 - days) / 10).
    availability = simulate_real_time_availability(metadata)
    if availability <= 0:
        availability_score = 0
    else:
        availability_score = max(0, (10 - availability) / 10)

    # Proximity Score: Compare provider and patient location.
    doc_location = metadata.get('location', '')
    if doc_location:
        doc_parts = doc_location.split(',')
        doc_city = doc_parts[0].strip().lower() if len(doc_parts) > 0 else ""
        doc_state = doc_parts[1].strip().lower() if len(doc_parts) > 1 else ""
    else:
        doc_city, doc_state = "", ""

    patient_parts = patient_location.split(',')
    patient_city = patient_parts[0].strip().lower() if len(patient_parts) > 0 else ""
    patient_state = patient_parts[1].strip().lower() if len(patient_parts) > 1 else ""

    if not doc_location:
        proximity_score = 0
    else:
        if doc_city == patient_city:
            proximity_score = 1
        elif doc_state == patient_state:
            proximity_score = 0.5
        else:
            proximity_score = 0

    # Composite score based on weighted factors.
    composite_score = (0.4 * specialty_score +
                       0.2 * insurance_score +
                       0.2 * availability_score +
                       0.2 * proximity_score)

    # Return both the score and a breakdown of individual factors.
    return composite_score, {
        "specialty_score": specialty_score,
        "insurance_score": insurance_score,
        "availability_score": availability_score,
        "proximity_score": proximity_score
    }

# RAG Pipeline

In [8]:
def rag_pipeline(patient_query: str, vector_store: FAISS,
                 patient_location="New Haven, CT", patient_insurance="BlueCross",
                 confidence_threshold: float = 0.6):
    """
    The RAG pipeline for matching specialists performs the following steps:
      1. Extracts the recommended specialty from the patient query.
      2. Executes a semantic search using that specialty.
      3. Calculates composite scores for candidate profiles.
      4. Sorts and selects the top 3 matches.
      5. Implements a reflection loop if top candidates score below the threshold.
      6. Returns a structured JSON with detailed, patient-friendly explanations.
    """
    # Step 1: Extract recommended specialty (using our advanced method).
    recommended_specialty = advanced_extract_specialty(patient_query)

    # Step 2: Form a basic semantic search query.
    search_query = f"Specialty: {recommended_specialty}"
    # Retrieve extra candidates to allow for re-ranking.
    search_results = vector_store.similarity_search(search_query, k=5)

    candidates = []
    for doc in search_results:
        composite, details = calculate_composite_score(doc.metadata, patient_location, patient_insurance, recommended_specialty)
        explanation = (
            f"Matched due to a specialty score of {details['specialty_score']}, "
            f"insurance score of {details['insurance_score']}, availability score of {details['availability_score']:.2f}, "
            f"and proximity score of {details['proximity_score']}."
        )
        candidate = {
            "name": doc.metadata.get('name', 'Unknown'),
            "specialty": doc.metadata.get('specialty', 'Unknown'),
            "composite_score": round(composite, 2),
            "explanation": explanation,
            "metadata": doc.metadata  # Optional metadata for debugging/integration.
        }
        candidates.append(candidate)

    # Sort the candidates by composite score in descending order.
    candidates = sorted(candidates, key=lambda x: x["composite_score"], reverse=True)
    top_candidates = candidates[:3]

    # Reflection loop: if all top candidate scores are below the threshold, re-query with a refined prompt.
    if all(c["composite_score"] < confidence_threshold for c in top_candidates):
        print("Low confidence in initial candidates. Re-querying with refined parameters...")
        refined_query = f"Expert in {recommended_specialty} with high availability and network compatibility"
        search_results = vector_store.similarity_search(refined_query, k=5)
        candidates = []
        for doc in search_results:
            composite, details = calculate_composite_score(doc.metadata, patient_location, patient_insurance, recommended_specialty)
            explanation = (
                f"After re-querying, this provider has a specialty score of {details['specialty_score']}, "
                f"insurance score of {details['insurance_score']}, availability score of {details['availability_score']:.2f}, "
                f"and proximity score of {details['proximity_score']}."
            )
            candidate = {
                "name": doc.metadata.get('name', 'Unknown'),
                "specialty": doc.metadata.get('specialty', 'Unknown'),
                "composite_score": round(composite, 2),
                "explanation": explanation,
                "metadata": doc.metadata
            }
            candidates.append(candidate)
        candidates = sorted(candidates, key=lambda x: x["composite_score"], reverse=True)
        top_candidates = candidates[:3]

    # Structure the final output JSON.
    output = {
         "patient_query": patient_query,
         "recommended_specialty": recommended_specialty,
         "patient_location": patient_location,
         "patient_insurance": patient_insurance,
         "specialist_matches": top_candidates
    }
    return output

# Final Call

In [9]:
# Sample patient free-text input.
patient_query = "I'm having chest pain during mild activity, it gets worse with exertion, and sometimes I feel dizzy."

# Execute the upgraded RAG pipeline.
result = rag_pipeline(patient_query, vector_store)

# Display the structured JSON output.
print("Referral Matching Agent Output:")
print(json.dumps(result, indent=4))

Referral Matching Agent Output:
{
    "patient_query": "I'm having chest pain during mild activity, it gets worse with exertion, and sometimes I feel dizzy.",
    "recommended_specialty": "Cardiologist",
    "patient_location": "New Haven, CT",
    "patient_insurance": "BlueCross",
    "specialist_matches": [
        {
            "name": "Dr. Aarti Mehta",
            "specialty": "Cardiologist",
            "composite_score": 0.96,
            "explanation": "Matched due to a specialty score of 1, insurance score of 1, availability score of 0.80, and proximity score of 1.",
            "metadata": {
                "specialist_id": 1,
                "name": "Dr. Aarti Mehta",
                "specialty": "Cardiologist",
                "location": "New Haven, CT",
                "insurance_accepted": "BlueCross, Aetna",
                "availability_next_3_days": 3,
                "profile_summary": "Board-certified cardiologist with 10 years of experience in treating arrhythmia a